## CEBD 1260 Final Programming Assignment
**Student: ** Deyvid William Angioletti Evaristo **ID: ** 6726364 

Complete both questions in a single jupyter notebook and upload to github repo in a new directory named "final".

Answers should include both code and written explanation / interpretation of results. Be sure to answer all parts of the question completely.

**1) Your first task to is to classify data from a cancer diagnostic database. In this database are patients with tumors, characteristics of those tumors, and biospy results indicating whether the tumor is Malignant or Benign.**


In cancer_data.txt you will find the following variables:

   - radius (mean of distances from center to points on the perimeter)
   - texture (standard deviation of gray-scale values)
   - perimeter
   - area
   - smoothness (local variation in radius lengths)
   - compactness (perimeter^2 / area - 1.0)
   - concavity (severity of concave portions of the contour)
   - concave_points (number of concave portions of the contour)
   - symmetry 
   - fractal_dimension ("coastline approximation" - 1)
   - cancer (0 = Benign, 1 = Malignant)  *target*


Use any machine learning algorithm you wish. In your answer include a short description of your algorithm of choice and predicted category of a new patient with a tumor with the following features:

   - radius: 14
   - texture: 14
   - perimeter: 88
   - area: 566
   - smoothness: 1
   - compactness: 0.08
   - concavity: 0.06
   - concae points: 0.04
   - symmetry: 0.18
   - fractal dimension: 0.05

In [679]:
import numpy as np
import pandas
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

# Load the cancer dataset
f = open("cancer_data.csv")
f.readline()  # skip the header
cancer_data = np.loadtxt(f, delimiter = ',')

# Load the test dataset (target not included)
f = open("cancer_test.csv")
f.readline()  # skip the header
cancer_test = np.loadtxt(f, delimiter = ',')
cancer_test = cancer_test.reshape(1, -1)

In [680]:
# Use 10 variables/features
# radius, texture, perimeter, area, smoothness, compactness, concavity, concae points,symmetry, fractal dimension
cancer_X = cancer_data[:,:10]

# Split the data into training/testing sets (X)
cancer_X_train = cancer_X[:]
cancer_X_test = cancer_test

# Specifying the targets for training set (Y)
cancer_y_train = cancer_data[:,10]

In [681]:
# Create a Linear classifier/logistic regression object
regr = linear_model.SGDClassifier(loss='log')

# Train the model using the training sets
regr.fit(cancer_X_train, cancer_y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [682]:
# Make predictions using the testing set
cancer_y_pred = regr.predict(cancer_test)

In [683]:
# The coefficients
print('Coefficients: \n', regr.coef_)

Coefficients: 
 [[  4.14833507e+03   6.18566597e+03   2.41063996e+04  -2.16961498e+03
    3.72265869e+01  -1.95288762e+01  -7.37959053e+01  -3.52752081e+01
    7.33272633e+01   3.10334287e+01]]


In [684]:
#Checking predicted category
print('Predicted category: \n',cancer_y_pred) 

Predicted category: 
 [ 1.]


The predicted cancer category is **1 (Malign type of Cancer) ** for a new patient with a tumor with the following features
   - radius: 14
   - texture: 14
   - perimeter: 88
   - area: 566
   - smoothness: 1
   - compactness: 0.08
   - concavity: 0.06
   - concae points: 0.04
   - symmetry: 0.18
   - fractal dimension: 0.05


**2) The following code contains/**had** 5 bugs (errors). Find and correct them all and then answer the following questions**

  1. How many observations are in the training dataset?

  2. How many features are in the training dataset?

  3. How well did your model perform?

**  BONUS: Which category is Hockey? 0 or 1? Which category is baseball? **

In [685]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier

categories = [ 'rec.sport.baseball','rec.sport.hockey']
twenty_train = fetch_20newsgroups(subset='train', categories=categories)
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, random_state=42)),
])
text_clf.fit(twenty_train.data, twenty_train.target)  
predicted = text_clf.predict(twenty_test.data)

In [686]:
#Observations in the training dataset
print(len(twenty_train.data))

1197


** There are 1,197 observations in the training dataset **

In [687]:
#How many features are in the training dataset?
print(len(text_clf.named_steps['clf'].coef_[0]))

18571


** There are 18,571 features in the training dataset **

In [688]:
print('Confirming observations/features numbers: \n', text_clf.named_steps['vect'].fit_transform(twenty_train.data).shape)

Confirming observations/features numbers: 
 (1197, 18571)


In [689]:
#How well did your model perform?

# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(twenty_test.target, predicted))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(twenty_test.target, predicted))


Mean squared error: 0.03
Variance score: 0.87


The model performed very well
 - Mean squared error is low
 - Variance score is very close to 1

In [690]:
#Which category is Hockey? 0 or 1? Which category is baseball?

#Looking at target names within the dataset
pandas.DataFrame(twenty_test.target_names)

,0
0,rec.sport.baseball
1,rec.sport.hockey


In [691]:
#Using prediction
print('Baseball: ',text_clf.predict(["Baseball"]))
print('Hockey:',text_clf.predict(["Hockey"]))

Baseball:  [0]
Hockey: [1]


** Baseball is category=0 and Hokey is category=1 **